Define initial variables here

In [ ]:
# run in terminal
mkdir /home/jupyter/github_reproducibility/TF_sensitive_peaks/

In [ ]:
output_name='/home/jupyter/github_reproducibility/TF_sensitive_peaks/TF_sensitive_peaks_new_definition.bed'


Importing libraries

In [ ]:
library(GenomicRanges)
library(data.table)
library(dplyr)
library(SummarizedExperiment)
library(Matrix)
library(diffloop)
library(rtracklayer)
#BiocParallel::register(BiocParallel::MulticoreParam(4, progressbar = FALSE))

`%notin%` <- Negate(`%in%`)

In [ ]:
combined_DAP_JAN_028_ATAC=fread("~/github_reproducibility/table_S2.csv")
all_gene_ids=unique(combined_DAP_JAN_028_ATAC %>%
filter(p_weight < 0.05 / 230000)%>%
pull(gene_ID))


In [ ]:
length(all_gene_ids)

In [ ]:
output_file <- output_name

data=all_gene_ids

data=data.frame(data)
colnames(data)='V2'

bed_data <- data.frame(Chromosome=character(), Start=integer(), End=integer(), stringsAsFactors=FALSE)

for (i in 1:nrow(data)) {
  splits <- unlist(strsplit(data$V2[i], "-"))
  bed_data <- rbind(bed_data, data.frame(Chromosome=splits[1], Start=as.integer(splits[2]), End=as.integer(splits[3]), stringsAsFactors=FALSE))
}

write.table(bed_data, output_file, sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE)
head(bed_data)

In [ ]:
bed_df <- read.table(output_name, header = FALSE, stringsAsFactors = FALSE)
bed_df <- bed_df[!is.na(bed_df$V3), ]
output_file <- output_name
write.table(bed_df, output_file, sep = "\t", quote = FALSE, row.names = FALSE, col.names = FALSE)



Copy the file and upload to the UCSC liftover website (https://genome.ucsc.edu/cgi-bin/hgLiftOver). After running it, copy the file back here.

In [ ]:
bed_df <- read.table('~/github_reproducibility/TF_sensitive_peaks/liftover/liftover_to_hg19_TF_sensitive_peaks_new_definition.bed', header = FALSE, stringsAsFactors = FALSE)

granges_object <- GRanges(seqnames=bed_df$V1,
                          ranges=IRanges(start=bed_df$V2, end=bed_df$V3))

print(granges_object)

The following code is slightly adapted from the github repo scLDSC/code/bed2annotate.R

In [ ]:
library(GenomicRanges)
library(data.table)

BiocParallel::register(BiocParallel::MulticoreParam(2, progressbar = FALSE))

makeAnnotFileFromBed <- function(chr, bedfile, outname = NULL, gzipIn = TRUE){

  if(is.null(outname) & gzipIn) outname <- gsub('.{7}$', '',basename(bedfile))
  if(is.null(outname) & !gzipIn) outname <- gsub('.{3}$', '',basename(bedfile))
  
  rawfile <- paste0("/home/jupyter/utilities/scLDSC/raw.annot/raw.", as.character(chr), ".annot.gz") 
  print(head(rawfile))
df <- data.frame(fread(input = paste0('zcat < ', rawfile)))
  gdf <- makeGRangesFromDataFrame(data.frame(chr = df$CHR, start = df$BP, end = df$BP))
  print(gdf)
  if(gzipIn){  beddf <- data.frame(fread(input =  bedfile))
  } else {  beddf <- data.frame(fread(input = paste0('', bedfile))) } 
  gbed <- makeGRangesFromDataFrame(data.frame(chr = beddf$V1, start = beddf$V2, end = beddf$V3))
    print(gbed)
    gdf_no_chr=gdf
seqlevels(gdf_no_chr) <- sub("chr", "", seqlevels(gdf))
    
        gbed_no_chr=gbed
seqlevels(gbed_no_chr) <- sub("chr", "", seqlevels(gbed))

  boo <- as.numeric(1:length(gdf) %in% queryHits(findOverlaps(gdf_no_chr, gbed_no_chr)))
  
  dat <- data.frame(df, boo)
  names(dat) <- c(colnames(df), outname)

  gz1 <- gzfile(paste0("/home/jupyter/github_reproducibility/TF_sensitive_peaks/custom_annot", as.character(chr), ".annot.gz"), "w")
  write.table(dat, file = gz1, row.names = FALSE, col.names = TRUE, sep = " ", quote = FALSE)
  close(gz1)
  return(chr)
}

baselineBeds <- list.files("/home/jupyter/github_reproducibility/TF_sensitive_peaks/liftover", full.names = TRUE)
baselineBeds
sapply(baselineBeds[1], function(bedfile){
  sapply(1:22, function(i){
    makeAnnotFileFromBed(i, bedfile)
    paste0(bedfile, "_", as.character(i))
  })
})

Remove the base annotation to avoid errors in the next step

In [ ]:
DEST_FOLDER="/home/jupyter/github_reproducibility/ldsc_analysis_TF_sensitive_peaks_NO_BASE"

mkdir -p "$DEST_FOLDER"

folder3="/home/jupyter/github_reproducibility/TF_sensitive_peaks/"

cd $folder3

for i in {1..22}; do
    gzip -d -c custom_annot$i.annot.gz | awk '{for(j=1;j<=NF;j++) if(j!=5) printf("%s%s",$j,(j==NF)?"\n":OFS)}' | gzip > "$DEST_FOLDER/custom_annot$i.annot.gz"
done

Run WDL (WDL_LDSC)